# Computer-Use Agents SOTA Challenge

Congrats on joining the Cua + HUD hackathon at Hack The North 2025!

This notebook will show you how to create a computer use agent with Cua and evaluate it using HUD.

## 💻 Prequisites

Clone the Cua repository and install project dependencies.

The easiest way to get started is by getting set up with the Cua development repository.

First, clone the Cua repository:

`git clone https://github.com/trycua/cua`

Install [pdm](https://pdm-project.org/en/latest/#recommended-installation-method).

Install the project dependencies:

`cd cua && pdm install`

Now, you should be able to run the `notebooks/hud_hackathon.ipynb` notebook in VS Code with the `.venv` virtual environment selected.

## ☁️ Connect to cloud services

Create Cua and HUD accounts and load your API keys. 

1. Create a Cua account at https://www.trycua.com/
2. Start a small Cua container at https://www.trycua.com/dashboard/containers (If you need credits, ask us!)
3. Create a HUD account at https://www.hud.so/
4. Create a .env file:

In [ ]:
# Create a .env file if it doesn't exist

ENV_TEMPLATE = """# Required environment variables:
CUA_API_KEY=
CUA_CONTAINER_NAME=
HUD_API_KEY=

# Any LLM provider will work:
ANTHROPIC_API_KEY=
OPENAI_API_KEY=
"""

import os
if not os.path.exists(".env"):
    open(".env", "w").write(ENV_TEMPLATE)
    print("A .env file was created! Fill in the empty values.")

5. Fill in all missing values in the .env file

In [2]:
# Read the .env file
# HUD requires the .env file to be in the same directory

from dotenv import load_dotenv
load_dotenv(dotenv_path='.env', override=True)

assert os.getenv("CUA_API_KEY")
assert os.getenv("CUA_CONTAINER_NAME")
assert os.getenv("HUD_API_KEY")

In [3]:
import sys, subprocess
pkgs = [
    "datasets>=2.20.0",   # HF datasets
    "hud-python>=0.2.0",  # HUD SDK
    "pyarrow>=14",        # needed by datasets
]
subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", *pkgs])


0

## 🤖 Create a computer use agent

Create and a computer use agent using the Cua SDK.

In [ ]:
import logging
from pathlib import Path
from agent import ComputerAgent

# Here you can set the model and tools for your agent.
# Computer use models: https://www.trycua.com/docs/agent-sdk/supported-agents/computer-use-agents
# Composed agent models: https://www.trycua.com/docs/agent-sdk/supported-agents/composed-agents
# Custom tools: https://www.trycua.com/docs/agent-sdk/custom-tools
agent_config = {
    "model": "anthropic/claude-3-7-sonnet-20250219",
    "trajectory_dir": str(Path("trajectories")),
    "only_n_most_recent_images": 6,
    "verbosity": logging.WARNING
}

## 🖱️ Test your agent

Run your agent on a test scenario in a Cua cloud container.

Connect to an existing cloud container through the Cua SDK.

You can access the computer through VNC on the [Cua Dashboard](https://www.trycua.com/dashboard).

In [5]:
from computer import Computer, VMProviderType

# Connect to your existing cloud container
computer = Computer(
    os_type="linux",
    provider_type=VMProviderType.CLOUD,
    name=os.getenv("CUA_CONTAINER_NAME") or "",
    api_key=os.getenv("CUA_API_KEY"),
    verbosity=logging.INFO
)

agent_config["tools"] = [ computer ]

Try running the computer use agent on a simple task.

To view a replay of the agent's actions, upload the trajectory to the [trajectory viewer](https://www.trycua.com/trajectory-viewer).

Trajectories are saved in the format: `trajectories/YYYY-MM-DD_computer-use-pre_XXX`.

In [ ]:
import logging, os
from pathlib import Path
from dotenv import load_dotenv
from agent import ComputerAgent
from computer import Computer, VMProviderType

load_dotenv(".env", override=True)

computer = Computer(
    os_type="linux",
    provider_type=VMProviderType.CLOUD,
    name=os.getenv("CUA_CONTAINER_NAME") or "",
    api_key=os.getenv("CUA_API_KEY"),
    verbosity=logging.WARNING,
)

# Main changes made here!
INSTRUCTIONS = (
  "SILENT MODE: Do not narrate; only reply 'DONE' at the end.\n"
  "GOAL: Complete exactly the task with the fewest reliable steps.\n"
  "FOCUS: Always press Ctrl-L before typing any URL or query.\n"
  "NAVIGATION: Type exactly, then Enter immediately. Do NOT use arrow keys to pick suggestions.\n"
  "BROWSER: Prefer one tab; avoid window dragging/resizing. Accept simple cookie prompts to proceed.\n"
  "VERIFICATION: After navigation, read page title and first H1 to confirm you’re at the correct page.\n"
  "RECOVERY: If a step fails twice, Ctrl-L → retype → Enter once. If still wrong, reload once.\n"
  "SEARCH (only if no URL is provided): use precise terms; prefer results on the exact target domain.\n"
  "STOP: As soon as the specified success text or obvious completion state is visible, reply 'DONE'."
  "APP MENU RULES (Desktop apps like LibreOffice/Impress/Calc):\n"
  "- Prefer keyboard menus: e.g., Alt+T then O for Tools→Options; Alt+S for the Sheet menu in Calc.\n"
  "- In dialogs, use Tab/Shift-Tab to move focus. Space toggles checkboxes/radios. Enter activates the default button.\n"
  "- In numeric fields, press Ctrl+A, type the exact value, then press Enter to commit.\n"
  "- DIALOG ANTI-LOOP: If a click fails twice, switch to keyboard (Tab → Ctrl+A → type → Enter). Never repeat the same mouse click more than twice in a modal.\n"
  "LIBREOFFICE AUTOSAVE (Load/Save → General):\n"
  "1) Alt+T,O to open Options. Select 'Load/Save', then 'General'.\n"
  "2) Ensure 'Save AutoRecovery information every' is CHECKED (Space).\n"
  "3) Tab to the minutes field, Ctrl+A, type 3, press Enter.\n"
  "4) Alt+O to OK. Then immediately reopen Options and verify it still shows 3 minutes before stopping.\n"
  "CALC SHEET COPY/RENAME:\n"
  " - Rename current sheet: Alt+S,R → type new name → Enter.\n"
  " - Copy sheet: Alt+S,M → tick 'Copy' (Space) → Tab to 'Insert before' and select target sheet → Tab to 'New name' → Ctrl+A → type exact name → Enter.\n"
  " - Verify final sheet names and order before stopping.\n"
  "STOP CONDITION GUARD: Do not stop after changing a setting until you have reopened the dialog and verified the value; do not stop after rename/copy until all names and order match exactly.\n"
)


agent = ComputerAgent(
    model="anthropic/claude-3-7-sonnet-20250219",
    tools=[computer],
    instructions=INSTRUCTIONS,
    use_prompt_caching=True,
    trajectory_dir=str(Path("trajectories")),
    only_n_most_recent_images=6,
    verbosity=logging.INFO,
)


tests = [
    "Open the browser and go to https://github.com/trycua/cua . ",
    "Success = the on-screen text 'README.md' is visible on the page.",
]

for t in tests:
    print(f"\nTask: {t}")
    async for _ in agent.run(t):
        pass
    print("✅ Done")



Task: Open the browser and go to https://github.com/trycua/cua . 


2025-09-14 03:50:36,948 - agent.ComputerAgent - INFO - LLM processing started with 2 messages
2025-09-14 03:50:39,809 - agent.ComputerAgent - INFO - Agent: I'll help you open a browser and navigate to that GitHub repository.
2025-09-14 03:50:39,811 - agent.ComputerAgent - INFO - Computer: screenshot({})
2025-09-14 03:50:41,572 - agent.ComputerAgent - INFO - LLM processing started with 5 messages
2025-09-14 03:50:45,226 - agent.ComputerAgent - INFO - Computer: click({'button': 'left', 'x': 536, 'y': 744})
2025-09-14 03:50:47,905 - agent.ComputerAgent - INFO - LLM processing started with 7 messages
2025-09-14 03:50:51,688 - agent.ComputerAgent - INFO - Computer: click({'button': 'left', 'x': 536, 'y': 684})
2025-09-14 03:50:53,251 - agent.ComputerAgent - INFO - LLM processing started with 9 messages
2025-09-14 03:50:57,586 - agent.ComputerAgent - INFO - Computer: keypress({'keys': ['ctrl', 'l']})
2025-09-14 03:50:59,164 - agent.ComputerAgent - INFO - LLM processing started with 11 messag

✅ Done

Task: Success = the on-screen text 'README.md' is visible on the page.


2025-09-14 03:51:41,984 - agent.ComputerAgent - INFO - LLM processing started with 2 messages
2025-09-14 03:51:46,370 - agent.ComputerAgent - INFO - Agent: I'll help you complete this task efficiently.
2025-09-14 03:51:46,370 - agent.ComputerAgent - INFO - Computer: screenshot({})
2025-09-14 03:51:48,205 - agent.ComputerAgent - INFO - LLM processing started with 5 messages
2025-09-14 03:52:00,936 - agent.ComputerAgent - INFO - Agent: I can see we're currently on the GitHub repository for trycua/cua. I need to navigate to view the README.md file.
2025-09-14 03:52:00,942 - agent.ComputerAgent - INFO - Computer: click({'button': 'left', 'x': 704, 'y': 657})
2025-09-14 03:52:03,084 - agent.ComputerAgent - INFO - LLM processing started with 8 messages
2025-09-14 03:52:08,349 - agent.ComputerAgent - INFO - Agent: I can now see the README.md content is visible on the page.

DONE
2025-09-14 03:52:09,085 - agent.ComputerAgent - INFO - Total usage:
 - input_tokens: 10213
 - output_tokens: 190
 -

✅ Done


## 🧐 Benchmark your agent

Test your agent's performance on a selection of tasks from the OSWorld benchmark.

In [ ]:
import uuid
from pprint import pprint
from agent.integrations.hud import run_full_dataset

job_name = f"osworld-test-{str(uuid.uuid4())[:4]}"

# Full dataset evaluation (runs via HUD's run_dataset under the hood)
# See the documentation here: https://docs.trycua.com/docs/agent-sdk/integrations/hud#running-a-full-dataset
results = await run_full_dataset(
    dataset="ddupont/OSWorld-Tiny-Public",
    job_name=job_name,
    **agent_config,
    max_concurrent=30,
    max_steps=95,
    #split="train[:5]"
)

# results is a list from hud.datasets.run_dataset; inspect/aggregate as needed
print(f"Job: {job_name}")
print(f"Total results: {len(results)}")
pprint(results[:3])

/Users/ayman/cua/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



╔════════════════════════════════════════════════════════════════╗
║               🚀 Job 'osworld-test-6ff3' started:              ║
╟────────────────────────────────────────────────────────────────╢
║  https://app.hud.so/jobs/f38687f4-cdfa-40a3-a12c-1d4df988579b  ║
╚════════════════════════════════════════════════════════════════╝



2025-09-14 03:54:37,983 - agent.ComputerAgent - INFO - LLM processing started with 2 messages
2025-09-14 03:54:42,657 - agent.ComputerAgent - INFO - LLM processing started with 2 messages
2025-09-14 03:54:44,283 - agent.ComputerAgent - INFO - LLM processing started with 2 messages
2025-09-14 03:54:44,968 - agent.ComputerAgent - INFO - LLM processing started with 2 messages
2025-09-14 03:54:47,159 - agent.ComputerAgent - INFO - LLM processing started with 2 messages
2025-09-14 03:54:47,795 - agent.ComputerAgent - INFO - LLM processing started with 5 messages
2025-09-14 03:54:57,034 - agent.ComputerAgent - INFO - LLM processing started with 2 messages
2025-09-14 03:54:58,207 - agent.ComputerAgent - INFO - LLM processing started with 2 messages
2025-09-14 03:54:58,945 - agent.ComputerAgent - INFO - LLM processing started with 5 messages
2025-09-14 03:55:02,409 - agent.ComputerAgent - INFO - LLM processing started with 5 messages
2025-09-14 03:55:03,145 - agent.ComputerAgent - INFO - LLM p

❌ Tool execution failed: Connection closed

2025-09-14 03:58:01,252 - agent.ComputerAgent - INFO - LLM processing started with 17 messages
2025-09-14 03:58:01,972 - agent.ComputerAgent - INFO - LLM processing started with 23 messages
2025-09-14 03:58:07,018 - agent.ComputerAgent - INFO - Agent: Great! I can now confirm that the Chrome profile username has been successfully changed to "Thomas". As you can see in the dropdown menu, the profile now shows "Thomas" instead of the previous name. The change has been applied and saved to your Chrome profile.

Is there anything else you'd like me to help you with regarding your Chrome profile or other settings?
2025-09-14 03:58:08,231 - agent.ComputerAgent - INFO - LLM processing started with 2 messages
2025-09-14 03:58:08,936 - agent.ComputerAgent - INFO - LLM processing started with 26 messages
2025-09-14 03:58:10,157 - agent.ComputerAgent - INFO - LLM processing started with 2 messages
2025-09-14 03:58:12,244 - agent.ComputerAgent - INFO - LLM processing started with 23 messages


...

2025-09-14 04:07:53,642 - agent.ComputerAgent - INFO - LLM processing started with 65 messages


❌ Tool execution failed: 

2025-09-14 04:07:56,267 - agent.ComputerAgent - INFO - LLM processing started with 65 messages
2025-09-14 04:07:56,981 - agent.ComputerAgent - INFO - LLM processing started with 41 messages


❌ Tool execution failed: 

2025-09-14 04:08:02,539 - agent.ComputerAgent - INFO - LLM processing started with 79 messages


## 🦾 Improve your agent

To improve your agent for OSWorld-Verified, experiment with different models and add custom tools that fit your use case. You can also dive into the ComputerAgent source code to design an improved version or subclass tailored to your needs.

Learn more about [Customizing Your ComputerAgent](https://docs.trycua.com/docs/agent-sdk/customizing-computeragent) in the docs.